In [1]:
#Setup
import pandas as pd
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<p style="font-family: Montserrat; color: #203443">- What are the available notification methods?</p>

<p style="font-family: Montserrat; color: #203443">- How do we use the Notifications API via the LUSID Website to surface data from an event in our email notifications via templating?</p>

<p style="font-family: Montserrat; color: #203443">- How can we use the Notifications API via the Python SDK to update our email notification to use HTML instead of plain text?</p>

<p style="font-family: Montserrat; color: #203443">- How can we use the Notifications API via the Python SDK to configure a text message notification?</p>

<p style="font-family: Montserrat; color: #203443">- How can we use the Notifications API via the Python SDK to delete a notification?</p>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What are the available notification methods?</p>

<p style="font-family: Montserrat; color: #203443">- Email</p>

<p style="font-family: Montserrat; color: #203443">- Text Message</p>

<p style="font-family: Montserrat; color: #203443">- Webhook</p>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Updating our email notification to use HTML</p>

In [2]:
import pprint

import lusid_notifications
from lusidjam import RefreshingToken
from fbnsdkutilities import ApiClientFactory

api_factory = ApiClientFactory(lusid_notifications, token=RefreshingToken())
notifications_api = api_factory.build(lusid_notifications.api.NotificationsApi)

subscription_scope = "FinbourneUniversity"
subscription_code = "AnyPortfolioCreated"

In [3]:
subscriptions_api = api_factory.build(lusid_notifications.api.SubscriptionsApi)

subscriptions_api.create_subscription(
    create_subscription=lusid_notifications.CreateSubscription(
        id=lusid_notifications.ResourceId(
            scope=subscription_scope,
            code=subscription_code,
        ),
        display_name="Any Portfolio Created",
        description="Listens to all TransactionPortfolioCreated events",
        status="Active",
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type="TransactionPortfolioCreated",
            filter=None
        )
    )
)

notifications_api.create_email_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_email_notification=lusid_notifications.CreateEmailNotification(
        description="Portfolio Created Email Notification",
        subject="New Portfolio Created!",
        plain_text_body="""
            A new Porfolio called {{PortfolioDisplayName}} has been created 
            with an identifier of {{PortfolioScope}} / {{PortfolioCode}} 
            and a creation date of {{PortfolioCreated}}
        """,
        html_body=None,
        email_address_to=["myemail@mycompany.com"],
        email_address_cc=[],
        email_address_bcc=[]
    )
)

{'content': {'HtmlBody': None,
             'PlainTextBody': '\n'
                              '            A new Porfolio called '
                              '{{PortfolioDisplayName}} has been created \n'
                              '            with an identifier of '
                              '{{PortfolioScope}} / {{PortfolioCode}} \n'
                              '            and a creation date of '
                              '{{PortfolioCreated}}\n'
                              '        ',
             'Subject': 'New Portfolio Created!'},
 'created_at': datetime.datetime(2023, 2, 9, 5, 58, 9, 728315, tzinfo=tzlocal()),
 'created_by': '00u89xerfkS08fj6e2p7',
 'delivery_channel': 'Email',
 'description': 'Portfolio Created Email Notification',
 'id': '6b53a658-bebc-4a58-b8cb-17e7852e82b2',
 'last_modified_at': datetime.datetime(2023, 2, 9, 5, 58, 9, 728315, tzinfo=tzlocal()),
 'last_modified_by': '00u89xerfkS08fj6e2p7',
 'recipients': {'EmailAddressBcc': [],
       

In [4]:
response = notifications_api.list_notifications(
    scope=subscription_scope,
    code=subscription_code)

pprint.pprint(response)

email_notification = response.values[0]

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://demo.lusid.com/app/insights/logs/0HMO8AHJNECDK:00000002',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'next_page': None,
 'previous_page': None,
 'values': [{'content': {'HtmlBody': None,
                         'PlainTextBody': '\n'
                                          '            A new Porfolio called '
                                          '{{PortfolioDisplayName}} has been '
                                          'created \n'
                                          '            with an identifier of '
                                          '{{PortfolioScope}} / '
                                          '{{PortfolioCode}} \n'
                                          '            and a creation date of '
                                          '{{Portfolio

In [5]:
with open("./data/EmailTemplate.html", 'r') as file:
    html_template = file.read()

print(html_template)

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" style="width:100%;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%;padding:0;Margin:0">
<head>
    <meta charset="UTF-8">
    <meta content="width=device-width, initial-scale=1" name="viewport">
    <meta name="x-apple-disable-message-reformatting">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta content="telephone=no" name="format-detection">
    <title>New message</title><!--[if (mso 16)]>
    <style type="text/css">
        a {text-decoration: none;}
    </style>
    <![endif]--><!--[if gte mso 9]><style>sup { font-size: 100% !important; }</style><![endif]--><!--[if gte mso 9]>
    <xml>
        <o:OfficeDocumentSettings>
            <o:AllowPNG></o:AllowPNG>
            <o:PixelsPerInch>96</o:PixelsPerInch>
        </o:OfficeDocumentSettings>
    </xml>
    <![endif]-->
    <style type="text/css">
        #outlook a {
            padding:0;
        }
        .Exte

In [6]:
updated_response = notifications_api.update_email_notification(
    scope=subscription_scope,
    code=subscription_code,
    id=email_notification.id,
    update_email_notification=lusid_notifications.UpdateEmailNotification(
        description=email_notification.description,
        subject=email_notification.content["Subject"],
        plain_text_body=email_notification.content["PlainTextBody"],
        html_body=html_template,
        email_address_to=email_notification.recipients["EmailAddressTo"],
        email_address_cc=email_notification.recipients["EmailAddressCc"],
        email_address_bcc=email_notification.recipients["EmailAddressBcc"]
    )
)

pprint.pprint(updated_response)

{'content': {'HtmlBody': '<html xmlns="http://www.w3.org/1999/xhtml" '
                         'xmlns:o="urn:schemas-microsoft-com:office:office" '
                         'style="width:100%;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%;padding:0;Margin:0">\n'
                         '<head>\n'
                         '    <meta charset="UTF-8">\n'
                         '    <meta content="width=device-width, '
                         'initial-scale=1" name="viewport">\n'
                         '    <meta '
                         'name="x-apple-disable-message-reformatting">\n'
                         '    <meta http-equiv="X-UA-Compatible" '
                         'content="IE=edge">\n'
                         '    <meta content="telephone=no" '
                         'name="format-detection">\n'
                         '    <title>New message</title><!--[if (mso 16)]>\n'
                         '    <style type="text/css">\n'
                         '  

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Creating a text message notification</p>

In [7]:
create_response = notifications_api.create_sms_notification(
    scope=subscription_scope,
    code=subscription_code,
    create_sms_notification=lusid_notifications.CreateSmsNotification(
        description="Portfolio Created",
        body="""A Portfolio called {{PortfolioDisplayName}} has been created 
            wtih id {{PortfolioScope}}/{{PortfolioCode}}.
            
            Please ensure it has been configured correctly by visiting:
            
            https://demo.lusid.com/app/dashboard/holdings?scope={{PortfolioScope}}&code={{PortfolioCode}}&entityType=Portfolio""",
        recipients=[
            "+447700900000",
            "+12334550000"
        ]
    )
)

pprint.pprint(create_response)

{'content': {'Body': 'A Portfolio called {{PortfolioDisplayName}} has been '
                     'created \n'
                     '            wtih id '
                     '{{PortfolioScope}}/{{PortfolioCode}}.\n'
                     '            \n'
                     '            Please ensure it has been configured '
                     'correctly by visiting:\n'
                     '            \n'
                     '            '
                     'https://demo.lusid.com/app/dashboard/holdings?scope={{PortfolioScope}}&code={{PortfolioCode}}&entityType=Portfolio'},
 'created_at': datetime.datetime(2023, 2, 9, 5, 58, 25, 334084, tzinfo=tzlocal()),
 'created_by': '00u89xerfkS08fj6e2p7',
 'delivery_channel': 'SMS',
 'description': 'Portfolio Created',
 'id': '8c2cde88-ba98-433b-864b-13064b9785ba',
 'last_modified_at': datetime.datetime(2023, 2, 9, 5, 58, 25, 334084, tzinfo=tzlocal()),
 'last_modified_by': '00u89xerfkS08fj6e2p7',
 'recipients': {'Recipients': ['+44770090

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Deleting a notification</p>

In [8]:
delete_response = notifications_api.delete_notification(
    scope=subscription_scope,
    code=subscription_code,
    id=create_response.id
)

pprint.pprint(delete_response)

None


In [9]:
subscriptions_api.delete_subscription(
    scope=subscription_scope,
    code=subscription_code
)


<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What have we covered?</p>

<p style="font-family: Montserrat; color: #203443">- There are three available notification methods. Email, text message and webhook.</p>

<p style="font-family: Montserrat; color: #203443">- You can use templating to surface data from the event in the notification. We set this up for an email notification using the Notifications API via the LUSID website.</p>

<p style="font-family: Montserrat; color: #203443">- You can also send an email in HTML, we set this up by updating our email notification using the Notifications API via the Python SDK.</p>

<p style="font-family: Montserrat; color: #203443">- You can also use templating for text message notifications. We created such a notification using the Notifications API via the Python SDK.</p>

<p style="font-family: Montserrat; color: #203443">- Notifications can be deleted from a subscription. We deleted the text message notification using the Notifications API via the Python SDK.</p>